# Columnar data analysis with `DAOD_PHYSLITE`

* prototype format within ATLAS to provide a small (~10kb/event), generic analysis format for end-user analysis
* standard set of calibrations already applied during production  
  suitable for fast downstream processing

It is one of the components of the ATLAS analysis model starting from Run-3:

<img src="img/run3_model_focus.png" width="800"/>

(plot from [presentation at CHEP2020](https://doi.org/10.1051/epjconf/202024506014))


The format and corresponding analysis applications are still under development

<div class="alert alert-block alert-success">
→ focus of this presentation: columnar data analysis with python tools on this format
</div>

For further information also see

- [VCHEP2021 presentation](https://doi.org/10.1051/epjconf/202125103001)
- code for columnar analysis R&D studies: https://gitlab.cern.ch/nihartma/physlite-experiments

## Reading the data using uproot

The PHYSLITE ROOT files currently follow a similar structure as regular ATLAS xAODs

They containing several trees, where the one holding the actual data is called `CollectionTree`. The others contain various forms of Metadata.

In [ ]:
import uproot

In [ ]:
f = uproot.open("data/DAOD_PHYSLITE_21.2.108.0.art.pool.root")

In [ ]:
f.keys()

### 1-D vectors
* All branches are stored with the **highest split level**
* In most cases data stored in branches called `Aux.<something>` or `AuxDyn.<something>`
* Typically **vectors of fundamental types**, like e.g. pt/eta/phi of particle collections
* **can be read into numpy arrays efficiently using uproot** since data stored as contiguous blocks  
(except for the 10-byte vector headers whoose positions are known from ROOT's event offsets)

In [ ]:
f["CollectionTree"].show("/AnalysisElectronsAuxDyn.(pt|eta|phi)$/i", name_width=30, interpretation_width=50)

### ElementLinks

The most relevant exception to this: `ElementLink` branches:

* provide cross references into other collections
* **often 2-dimensional** (`vector<vector<ElementLink<...>>>`)
* data part (`ElementLink`) is serialized as a **structure of 2 32bit unsigned integers**:
  * hash `m_persKey`, identifying the target collection
  * index `m_persIndex` identifying the array-index of the corresponding particle in the target collection.

In [ ]:
f["CollectionTree/AnalysisElectronsAuxDyn.trackParticleLinks"].typename

In [ ]:
[element.all_members for element in f.file.streamer_named("ElementLinkBase").elements]

Uproot can read this, but the loop that deserializes the data is done in python and therefore slow.

This is not relevant for this very small file, but becomes important for larger files.

In [ ]:
%%time
f["CollectionTree/AnalysisElectronsAuxDyn.trackParticleLinks"].array()

This can be handled by [AwkwardForth](https://doi.org/10.1051/epjconf/202125103002) which is however currently (November 2021) not yet integrated with uproot.

I included a small module that can handle the relevant branches in PHYSLITE with a function `branch_to_array` that uses AwkwardForth internally.

In [ ]:
from awkward_forth_physlite import branch_to_array

In [ ]:
branch_to_array(f["CollectionTree/AnalysisElectronsAuxDyn.trackParticleLinks"])

One can actually see a significant improvement already for the small file with only 40 events!

In [ ]:
%%timeit
# using standard uproot
f.file.array_cache.clear()
f["CollectionTree/AnalysisElectronsAuxDyn.trackParticleLinks"].array()

In [ ]:
%%timeit
# using awkward forth
f.file.array_cache.clear()
branch_to_array(f["CollectionTree/AnalysisElectronsAuxDyn.trackParticleLinks"])

## Integration with `coffea.nanoevents`

The PHYSLITE schema and the corresponding behavior classes are still under development - [CoffeaTeam/coffea#540](https://github.com/CoffeaTeam/coffea/issues/540) tracks the progress of some TODO items.

For more information on `NanoEvents` see the [NanoEvents tutorial](https://github.com/CoffeaTeam/coffea/blob/master/binder/nanoevents.ipynb) or [Nick Smith's presentation](https://youtu.be/udzkE6t4Mck) at the [pyHEP 2020](https://indico.cern.ch/event/882824).

<div class="alert alert-block alert-success">
    <b>The Goal:</b>
    <ul>
        <li>Work with object-oriented event data models, but stick to the array-at-a-time processing paradigm.<br> → Struct/Object of arrays instead of Array of structs/objects</li>
        <li>Hide the details from the user</li>
    </ul>
</div>

### The NanoEvents object

In [ ]:
# uncomment this for reading larger files
# (needed for good performance until AwkwardForth is integrated with uproot)

#from awkward_forth_physlite import patch_nanoevents
#patch_nanoevents()

In [ ]:
from coffea.nanoevents import NanoEventsFactory, PHYSLITESchema

In [ ]:
factory = NanoEventsFactory.from_root(
    "data/DAOD_PHYSLITE_21.2.108.0.art.pool.root", "CollectionTree", schemaclass=PHYSLITESchema
)
events = factory.events()

In [ ]:
events

In [ ]:
events.Electrons

All columns from the `Aux` and `AuxDyn` branches are available and automatically grouped under the collections:

In [ ]:
events.Electrons.fields

In [ ]:
events.Electrons.pt

### Lazy loading

Data is read lazily. Columns are only loaded from disk once requested! After loading once, they are cached in memory.

We can set a debug flag to test the lazy loading:

In [ ]:
from coffea.nanoevents.mapping import UprootSourceMapping
UprootSourceMapping._debug = True

In [ ]:
events.Muons.pt

The second time, the array is cached:

In [ ]:
events.Muons.pt

In [ ]:
UprootSourceMapping._debug = False

Most collections have LorentzVector behavior and can utilize the methods available in coffea:

In [ ]:
# invariant mass of first and second jet in each event
(events.Jets[:, 0] + events.Jets[:, 1]).mass

In [ ]:
# delta_r between each Electron and it's closest Jet
closest_jets = events.Electrons.nearest(events.Jets)
events.Electrons.delta_r(closest_jets)

Cross references work transparently:

In [ ]:
events.Electrons.trackParticles

In [ ]:
events.Electrons.trackParticles.z0

What happened here?

- read event offsets from one column of the TrackParticle collection
- read one column of the Electron collection to get the event index of each Electron
- read Electron-TrackParticle ElementLinks
- read requested column from TrackParticle collection
- generate global index (into flat TrackParticle array) from event index and local index stored in ElementLink  
  → slightly different approach as in CMS NanoAOD
  - NanoAOD: global index created from target collection
  - PHYSLITE: global index created from event index + local index
  - Advantage: don't need to know target collection beforehand
    (in PHYSLITE the target is stored per-link in the `m_persKey` property)
- create IndexedArray from this

In [ ]:
events._caches[0].clear()
UprootSourceMapping._debug = True
print(events.Electrons.trackParticles.z0)
UprootSourceMapping._debug = False

In [ ]:
events.Electrons.trackParticles.z0.layout.form

Working with global indices allows cross referencing even when the array is sliced, selected or reshuffled:

In [ ]:
events.Electrons[events.Electrons.pt > 10000].trackParticles

In [ ]:
events[[2, 3]].Electrons.trackParticles.pt.tolist()

In [ ]:
events[[3, 2]].Electrons.trackParticles.pt.tolist()

**Specific to PHYSLITE**: Can link into multiple target collections.

For example: TruthCollections potentially contain links into multiple other TruthCollections:

In [ ]:
# under the hood, this is a union array
events.TruthElectrons.parents

In [ ]:
events.TruthElectrons.parents.pdgId

behavior will be attached to the resulting arrays, so one can in principle also do "cyclic" references:

In [ ]:
events.TruthElectrons.parents.children.parents.children.pdgId

In [ ]:
events.TruthElectrons.parents.children.parents.children.pdgId.ndim

## Open questions

- How to handle systematics/more complicated things (like e.g. MET)?
  - Simplify application of systematics, e.g. parametrized for simple application?
  - Or can we provide an interface to existing C++ CP tools?
- How far can this analysis style be brought upstream?
  - Directly run on raw PHYSLITE content?
  - Or produce smaller ntuples?